In [1]:
# Cell 1: Imports and Configuration
import os
import sys
import warnings
from datetime import datetime
from pathlib import Path
import logging
import json
import pickle

import numpy as np
import pandas as pd
from scipy import stats

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

# Configuration
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8')
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.4f' % x)

print("✓ All imports successful")


✓ All imports successful


In [2]:
# Cell 2: Configuration Class
class ChurnConfig:
    """Centralized configuration for churn model preprocessing"""
    
    # Paths
    DATA_DIR = Path("data")
    OUTPUT_DIR = Path("output")
    LOG_DIR = Path("logs")
    FIGURES_DIR = Path("figures")
    
    # Data files
    TRAIN_INPUT_FILE = "churn_train_ul.csv"
    PROD_INPUT_FILE = "churn_prod_ul.csv"
    
    OUTPUT_FILE = "churn_data_preprocessed.csv"
    TRAIN_OUTPUT_FILE = "train_processed.csv"
    TEST_OUTPUT_FILE = "test_processed.csv"
    VAL_OUTPUT_FILE = "val_processed.csv"
    METADATA_FILE = "preprocessing_metadata.json"
    
    # Data loading
    DELIMITER = '|'
    ENCODING = 'windows-1251'
    
    # Preprocessing parameters
    CORRELATION_THRESHOLD = 0.85
    OUTLIER_IQR_MULTIPLIER = 1.5
    REMOVE_GAPS = True
    HANDLE_OUTLIERS = True
    REMOVE_HIGH_CORRELATIONS = True
    
    # Train/Val/Test split - TEMPORAL SPLIT
    TRAIN_SIZE = 0.70
    VAL_SIZE = 0.15
    TEST_SIZE = 0.15
    
    # Model parameters
    RANDOM_SEED = 42
    
    # Columns to exclude from preprocessing
    ID_COLUMNS = ['cli_code', 'client_id', 'observation_point']
    TARGET_COLUMN = 'target_churn_3m'
    
    # CatBoost categorical features
    CATEGORICAL_FEATURES = [
        'segment_group',
        'obs_month',
        'obs_quarter'
    ]
    
    # Feature groups
    FINANCIAL_KEYWORDS = ['profit', 'income', 'expense', 'margin', 'provision']
    BALANCE_KEYWORDS = ['balance', 'assets', 'liabilities']
    ACTIVITY_KEYWORDS = ['activity', 'active', 'months_with']
    
    @classmethod
    def create_directories(cls):
        """Create necessary directories"""
        for dir_path in [cls.DATA_DIR, cls.OUTPUT_DIR, cls.LOG_DIR, cls.FIGURES_DIR]:
            dir_path.mkdir(parents=True, exist_ok=True)
    
    @classmethod
    def get_train_input_path(cls):
        return cls.DATA_DIR / cls.TRAIN_INPUT_FILE
    
    @classmethod
    def get_prod_input_path(cls):
        return cls.DATA_DIR / cls.PROD_INPUT_FILE

# Initialize configuration
config = ChurnConfig()
config.create_directories()
np.random.seed(config.RANDOM_SEED)

print("✓ Configuration initialized")
print(f"  - Train data: {config.get_train_input_path()}")
print(f"  - Prod data: {config.get_prod_input_path()}")

✓ Configuration initialized
  - Train data: data\churn_train_ul.csv
  - Prod data: data\churn_prod_ul.csv


In [3]:
# Cell 3: Logging Setup
def setup_logging():
    """Setup logging configuration"""
    log_file = config.LOG_DIR / f"preprocessing_{datetime.now().strftime('%Y%m%d_%H%M%S')}.log"
    
    file_formatter = logging.Formatter(
        '%(asctime)s - %(name)s - %(levelname)s - %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S'
    )
    console_formatter = logging.Formatter('%(levelname)s - %(message)s')
    
    # File handler
    file_handler = logging.FileHandler(log_file, encoding='utf-8')
    file_handler.setLevel(logging.INFO)
    file_handler.setFormatter(file_formatter)
    
    # Console handler
    console_handler = logging.StreamHandler(sys.stdout)
    console_handler.setLevel(logging.INFO)
    console_handler.setFormatter(console_formatter)
    
    # Setup root logger
    logger = logging.getLogger()
    logger.setLevel(logging.INFO)
    
    # Clear existing handlers
    logger.handlers = []
    
    logger.addHandler(file_handler)
    logger.addHandler(console_handler)
    
    return logging.getLogger(__name__)

logger = setup_logging()
logger.info("="*60)
logger.info("CHURN MODEL PREPROCESSING PIPELINE STARTED")
logger.info("="*60)

print("✓ Logging configured")

INFO - ============================================================
INFO - CHURN MODEL PREPROCESSING PIPELINE STARTED
INFO - ============================================================
✓ Logging configured


In [4]:
# Cell 4: DataLoader with Memory Optimization
class DataLoader:
    """Handle data loading with validation and memory optimization"""
    
    def __init__(self, file_path, delimiter='|', encoding='utf-8'):
        self.file_path = Path(file_path)
        self.delimiter = delimiter
        self.encoding = encoding
        self.logger = logging.getLogger(self.__class__.__name__)
    
    def load(self, optimize_memory=True):
        """Load data with memory optimization"""
        try:
            self.logger.info(f"Loading data from: {self.file_path}")
            
            if not self.file_path.exists():
                raise FileNotFoundError(f"Data file not found: {self.file_path}")
            
            # Load with optimized dtypes
            df = pd.read_csv(
                self.file_path,
                delimiter=self.delimiter,
                encoding=self.encoding,
                thousands=',',
                low_memory=False
            )
            
            memory_before = df.memory_usage(deep=True).sum() / 1024**2
            self.logger.info(f"  Memory before optimization: {memory_before:.2f} MB")
            
            # Standardize column names
            df.columns = df.columns.str.lower().str.strip()
            
            # OPTIMIZE MEMORY
            if optimize_memory:
                df = self._optimize_dtypes(df)
                memory_after = df.memory_usage(deep=True).sum() / 1024**2
                savings = (1 - memory_after/memory_before) * 100
                self.logger.info(f"  Memory after optimization: {memory_after:.2f} MB ({savings:.1f}% saved)")
            
            self.logger.info(f"✓ Data loaded: {df.shape}")
            self._validate_data(df)
            
            return df
            
        except Exception as e:
            self.logger.error(f"✗ Error loading data: {e}")
            raise
    
    def _optimize_dtypes(self, df):
        """Optimize DataFrame dtypes to reduce memory"""
        for col in df.columns:
            col_type = df[col].dtype
            
            # Skip ID columns, target, and categorical features
            if col in config.ID_COLUMNS + [config.TARGET_COLUMN] + config.CATEGORICAL_FEATURES:
                continue
            
            # Optimize numeric columns
            if col_type != 'object':
                c_min = df[col].min()
                c_max = df[col].max()
                
                # Integer types
                if str(col_type)[:3] == 'int':
                    if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                        df[col] = df[col].astype(np.int8)
                    elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                        df[col] = df[col].astype(np.int16)
                    elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                        df[col] = df[col].astype(np.int32)
                
                # Float types
                else:
                    if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                        df[col] = df[col].astype(np.float32)
        
        return df
    
    def _validate_data(self, df):
        """Validate loaded data"""
        missing_id_cols = [col for col in config.ID_COLUMNS if col not in df.columns]
        if missing_id_cols:
            self.logger.warning(f"⚠ Missing ID columns: {missing_id_cols}")
        
        if df.empty:
            raise ValueError("Loaded dataframe is empty")
        
        if config.TARGET_COLUMN in df.columns:
            unique_targets = df[config.TARGET_COLUMN].unique()
            if not set(unique_targets).issubset({0, 1, np.nan}):
                self.logger.warning(f"⚠ Unexpected target values: {unique_targets}")
            self.logger.info("✓ Data validation passed (with target)")
        else:
            self.logger.info("✓ Data validation passed (no target - production data)")

print("✓ DataLoader class defined with memory optimization")


✓ DataLoader class defined with memory optimization


In [5]:
# Cell 5: Load Training Data
logger.info("\n" + "="*60)
logger.info("LOADING TRAINING DATA")
logger.info("="*60)

loader = DataLoader(config.get_train_input_path(), delimiter=config.DELIMITER)
train_full_df = loader.load(optimize_memory=True)

print(f"\n✓ Training data loaded: {train_full_df.shape}")
print(f"  Columns: {list(train_full_df.columns[:10])}...")
print(f"  Target distribution:\n{train_full_df[config.TARGET_COLUMN].value_counts()}")


INFO - 
INFO - LOADING TRAINING DATA
INFO - ============================================================
INFO - Loading data from: data\churn_train_ul.csv
INFO -   Memory before optimization: 5194.33 MB
INFO -   Memory after optimization: 2286.35 MB (56.0% saved)
INFO - ✓ Data loaded: (3243871, 195)
INFO - ✓ Data validation passed (with target)

✓ Training data loaded: (3243871, 195)
  Columns: ['cli_code', 'client_id', 'observation_point', 'target_churn_3m', 'segment_group', 'obs_months_count', 'avg_activity_6m', 'active_months_6m', 'avg_products_6m', 'max_products_6m']...
  Target distribution:
target_churn_3m
0    3195163
1      48708
Name: count, dtype: int64


In [6]:
# Cell 6: Data Quality Analysis Class & Execution
class DataQualityAnalyzer:
    """Analyze data quality and generate reports"""
    
    def __init__(self, df):
        self.df = df
        self.logger = logging.getLogger(self.__class__.__name__)
        self.quality_report = {}
    
    def analyze(self):
        """Run comprehensive data quality analysis"""
        self.logger.info("="*60)
        self.logger.info("DATA QUALITY ANALYSIS")
        self.logger.info("="*60)
        
        self._basic_info()
        self._check_missing_values()
        self._check_constant_columns()
        self._check_duplicates()
        self._analyze_target_distribution()
        self._analyze_temporal_distribution()
        
        return self.quality_report
    
    def _basic_info(self):
        info = {
            'shape': self.df.shape,
            'n_rows': len(self.df),
            'n_columns': len(self.df.columns),
            'memory_usage_mb': self.df.memory_usage(deep=True).sum() / 1024**2,
            'n_numeric': len(self.df.select_dtypes(include=[np.number]).columns),
            'n_categorical': len(self.df.select_dtypes(include=['object']).columns)
        }
        self.quality_report['basic_info'] = info
        self.logger.info(f"Shape: {info['shape']}, Memory: {info['memory_usage_mb']:.2f} MB")
    
    def _check_missing_values(self):
        missing = self.df.isnull().sum()
        missing_pct = (missing / len(self.df) * 100).round(2)
        missing_df = pd.DataFrame({
            'column': missing.index,
            'missing_count': missing.values,
            'missing_pct': missing_pct.values
        })
        missing_df = missing_df[missing_df['missing_count'] > 0].sort_values('missing_pct', ascending=False)
        self.quality_report['missing_values'] = missing_df
        
        if len(missing_df) > 0:
            self.logger.info(f"⚠ {len(missing_df)} columns with missing values")
        else:
            self.logger.info("✓ No missing values")
    
    def _check_constant_columns(self):
        constant_cols = []
        near_constant_cols = []
        
        for col in self.df.columns:
            n_unique = self.df[col].nunique(dropna=False)
            if n_unique == 1:
                constant_cols.append(col)
            elif n_unique == 2 and self.df[col].value_counts(normalize=True).iloc[0] > 0.99:
                near_constant_cols.append(col)
        
        self.quality_report['constant_columns'] = constant_cols
        self.quality_report['near_constant_columns'] = near_constant_cols
        
        if constant_cols:
            self.logger.info(f"ℹ {len(constant_cols)} constant columns")
        if near_constant_cols:
            self.logger.info(f"⚠ {len(near_constant_cols)} near-constant columns")
    
    def _check_duplicates(self):
        n_duplicates = self.df.duplicated().sum()
        self.quality_report['duplicates'] = {'count': n_duplicates}
        
        if n_duplicates > 0:
            self.logger.warning(f"⚠ {n_duplicates} duplicate rows")
        else:
            self.logger.info("✓ No duplicates")
    
    def _analyze_target_distribution(self):
        if config.TARGET_COLUMN not in self.df.columns:
            self.logger.info("ℹ No target column (production data)")
            self.quality_report['target_distribution'] = None
            return
        
        target_dist = self.df[config.TARGET_COLUMN].value_counts()
        churn_rate = self.df[config.TARGET_COLUMN].mean()
        
        self.quality_report['target_distribution'] = {
            'churn_rate': churn_rate,
            'n_churned': int(target_dist.get(1, 0)),
            'n_not_churned': int(target_dist.get(0, 0))
        }
        
        self.logger.info(f"Target: Churn rate = {churn_rate:.4f}")
    
    def _analyze_temporal_distribution(self):
        if 'observation_point' in self.df.columns:
            self.df['observation_point'] = pd.to_datetime(self.df['observation_point'])
            temporal_info = {
                'min_date': self.df['observation_point'].min(),
                'max_date': self.df['observation_point'].max(),
                'n_unique_dates': self.df['observation_point'].nunique(),
                'n_unique_clients': self.df['cli_code'].nunique()
            }
            self.quality_report['temporal_info'] = temporal_info
            self.logger.info(f"Period: {temporal_info['min_date'].date()} to {temporal_info['max_date'].date()}")

# Run quality analysis
quality_analyzer = DataQualityAnalyzer(train_full_df)
quality_report = quality_analyzer.analyze()

print("\n✓ Data quality analysis complete")

INFO - ============================================================
INFO - DATA QUALITY ANALYSIS
INFO - ============================================================
INFO - Shape: (3243871, 195), Memory: 2286.35 MB
INFO - ✓ No missing values
INFO - ℹ 9 constant columns
INFO - ⚠ 7 near-constant columns
INFO - ✓ No duplicates
INFO - Target: Churn rate = 0.0150
INFO - Period: 2023-06-30 to 2025-06-30

✓ Data quality analysis complete


In [7]:
# Cell 7: TEMPORAL SPLIT (Key Change - by time, not random!)
class TemporalSplitter:
    """Split data by TIME to prevent data leakage"""
    
    def __init__(self, df, date_col='observation_point'):
        self.df = df
        self.date_col = date_col
        self.logger = logging.getLogger(self.__class__.__name__)
    
    def split(self, train_size=0.70, val_size=0.15, test_size=0.15):
        """Split data temporally based on observation_point"""
        self.logger.info("="*60)
        self.logger.info("TEMPORAL TRAIN/VALIDATION/TEST SPLIT")
        self.logger.info("="*60)
        
        # Ensure date column is datetime
        if self.df[self.date_col].dtype != 'datetime64[ns]':
            self.df[self.date_col] = pd.to_datetime(self.df[self.date_col])
        
        # Sort by date
        df_sorted = self.df.sort_values(self.date_col).reset_index(drop=True)
        
        # Get unique dates
        unique_dates = sorted(df_sorted[self.date_col].unique())
        n_dates = len(unique_dates)
        
        self.logger.info(f"Total unique dates: {n_dates}")
        self.logger.info(f"Date range: {unique_dates[0].date()} to {unique_dates[-1].date()}")
        
        # Calculate cutoff indices based on proportions
        train_cutoff_idx = int(n_dates * train_size)
        val_cutoff_idx = int(n_dates * (train_size + val_size))
        
        # Get cutoff dates
        train_end_date = unique_dates[train_cutoff_idx - 1]
        val_end_date = unique_dates[val_cutoff_idx - 1]
        
        self.logger.info(f"\nSplit cutoff dates:")
        self.logger.info(f"  Train: up to {train_end_date.date()} ({train_cutoff_idx} dates)")
        self.logger.info(f"  Val: {unique_dates[train_cutoff_idx].date()} to {val_end_date.date()} ({val_cutoff_idx - train_cutoff_idx} dates)")
        self.logger.info(f"  Test: {unique_dates[val_cutoff_idx].date()} onwards ({n_dates - val_cutoff_idx} dates)")
        
        # Create splits
        train = df_sorted[df_sorted[self.date_col] <= train_end_date].copy()
        val = df_sorted[(df_sorted[self.date_col] > train_end_date) & 
                       (df_sorted[self.date_col] <= val_end_date)].copy()
        test = df_sorted[df_sorted[self.date_col] > val_end_date].copy()
        
        # Log statistics
        split_info = {
            'train': {
                'n_records': len(train),
                'n_clients': train['cli_code'].nunique(),
                'date_range': (train[self.date_col].min(), train[self.date_col].max())
            },
            'val': {
                'n_records': len(val),
                'n_clients': val['cli_code'].nunique(),
                'date_range': (val[self.date_col].min(), val[self.date_col].max())
            },
            'test': {
                'n_records': len(test),
                'n_clients': test['cli_code'].nunique(),
                'date_range': (test[self.date_col].min(), test[self.date_col].max())
            }
        }
        
        for name, info in split_info.items():
            self.logger.info(f"\n{name.upper()}:")
            self.logger.info(f"  Records: {info['n_records']}")
            self.logger.info(f"  Clients: {info['n_clients']}")
            self.logger.info(f"  Dates: {info['date_range'][0].date()} to {info['date_range'][1].date()}")
        
        # Verify no data leakage
        assert train[self.date_col].max() < val[self.date_col].min(), "Data leakage: train overlaps val!"
        assert val[self.date_col].max() < test[self.date_col].min(), "Data leakage: val overlaps test!"
        self.logger.info("\n✓ Temporal ordering verified - no data leakage")
        
        return train, val, test, split_info

logger.info("\n" + "="*60)
logger.info("CREATING TEMPORAL TRAIN/VAL/TEST SPLIT")
logger.info("="*60)

splitter = TemporalSplitter(train_full_df)
train_df, val_df, test_df, split_info = splitter.split(
    train_size=config.TRAIN_SIZE,
    val_size=config.VAL_SIZE,
    test_size=config.TEST_SIZE
)

print(f"\n✓ Temporal split complete")
print(f"  Train: {train_df.shape}")
print(f"  Val: {val_df.shape}")
print(f"  Test: {test_df.shape}")

INFO - 
INFO - CREATING TEMPORAL TRAIN/VAL/TEST SPLIT
INFO - ============================================================
INFO - ============================================================
INFO - TEMPORAL TRAIN/VALIDATION/TEST SPLIT
INFO - ============================================================
INFO - Total unique dates: 25
INFO - Date range: 2023-06-30 to 2025-06-30
INFO - 
Split cutoff dates:
INFO -   Train: up to 2024-10-31 (17 dates)
INFO -   Val: 2024-11-30 to 2025-02-28 (4 dates)
INFO -   Test: 2025-03-31 onwards (4 dates)
INFO - 
TRAIN:
INFO -   Records: 2162862
INFO -   Clients: 173448
INFO -   Dates: 2023-06-30 to 2024-10-31
INFO - 
VAL:
INFO -   Records: 535263
INFO -   Clients: 144022
INFO -   Dates: 2024-11-30 to 2025-02-28
INFO - 
TEST:
INFO -   Records: 545746
INFO -   Clients: 146264
INFO -   Dates: 2025-03-31 to 2025-06-30
INFO - 
✓ Temporal ordering verified - no data leakage

✓ Temporal split complete
  Train: (2162862, 195)
  Val: (535263, 195)
  Test: (545746,

In [8]:
# Cell 8: Gap Detection (AFTER split - only on train!)
class GapDetector:
    """Detect and handle temporal gaps - AFTER SPLIT to avoid leakage"""
    
    def __init__(self, df, client_col='cli_code', date_col='observation_point'):
        self.df = df
        self.client_col = client_col
        self.date_col = date_col
        self.logger = logging.getLogger(self.__class__.__name__)
    
    def detect_gaps(self):
        """Detect clients with gaps - CHUNKED VERSION"""
        self.logger.info("Analyzing temporal gaps (chunked)...")
        
        # Convert date once
        if self.df[self.date_col].dtype != 'datetime64[ns]':
            self.df[self.date_col] = pd.to_datetime(self.df[self.date_col])
        
        # Process in chunks by client
        unique_clients = self.df[self.client_col].unique()
        chunk_size = 10000
        clients_with_gaps_list = []
        
        for i in range(0, len(unique_clients), chunk_size):
            chunk_clients = unique_clients[i:i+chunk_size]
            
            chunk_df = self.df[self.df[self.client_col].isin(chunk_clients)].copy()
            chunk_df = chunk_df.sort_values([self.client_col, self.date_col])
            
            chunk_df['month_num'] = chunk_df[self.date_col].dt.to_period('M').apply(lambda x: x.ordinal)
            chunk_df['month_diff'] = chunk_df.groupby(self.client_col)['month_num'].diff()
            
            gaps_summary = chunk_df.groupby(self.client_col)['month_diff'].agg([
                ('max_gap', 'max'),
                ('total_gaps', lambda x: (x > 1).sum())
            ]).reset_index()
            
            chunk_clients_with_gaps = gaps_summary[gaps_summary['max_gap'] > 1]
            clients_with_gaps_list.append(chunk_clients_with_gaps)
            
            del chunk_df, gaps_summary
            
            if (i // chunk_size + 1) % 5 == 0:
                import gc
                gc.collect()
                self.logger.info(f"  Processed {i+chunk_size}/{len(unique_clients)} clients")
        
        clients_with_gaps = pd.concat(clients_with_gaps_list, ignore_index=True)
        
        gap_info = {
            'total_clients': len(unique_clients),
            'clients_with_gaps': len(clients_with_gaps),
            'pct_with_gaps': len(clients_with_gaps) / len(unique_clients) * 100
        }
        
        self.logger.info(f"Gaps: {gap_info['clients_with_gaps']} clients ({gap_info['pct_with_gaps']:.1f}%)")
        
        return clients_with_gaps, gap_info
    
    def remove_clients_with_gaps(self):
        """Remove clients with gaps"""
        clients_with_gaps, gap_info = self.detect_gaps()
        
        if len(clients_with_gaps) == 0:
            self.logger.info("✓ No gaps found")
            return self.df, gap_info
        
        bad_clients = set(clients_with_gaps[self.client_col].unique())
        
        original_len = len(self.df)
        mask = ~self.df[self.client_col].isin(bad_clients)
        df_clean = self.df[mask].copy()
        
        self.logger.info(f"✓ Removed {len(bad_clients)} clients with gaps")
        self.logger.info(f"  Records: {original_len} → {len(df_clean)}")
        
        del clients_with_gaps, bad_clients, mask
        import gc
        gc.collect()
        
        return df_clean, gap_info


if config.REMOVE_GAPS:
    logger.info("\n" + "="*60)
    logger.info("REMOVING TEMPORAL GAPS (TRAIN ONLY)")
    logger.info("="*60)
    
    gap_detector = GapDetector(train_df)
    train_df_new, gap_info = gap_detector.remove_clients_with_gaps()
    
    # Store list of clients to remove
    clients_to_remove = set(train_df['cli_code']) - set(train_df_new['cli_code'])
    
    # Also remove these clients from val and test
    if len(clients_to_remove) > 0:
        logger.info(f"\nRemoving {len(clients_to_remove)} clients from val/test as well...")
        val_df = val_df[~val_df['cli_code'].isin(clients_to_remove)].copy()
        test_df = test_df[~test_df['cli_code'].isin(clients_to_remove)].copy()
        logger.info(f"  Val: {len(val_df)} records remaining")
        logger.info(f"  Test: {len(test_df)} records remaining")
    
    del train_df
    train_df = train_df_new
    del train_df_new, clients_to_remove
    
    import gc
    gc.collect()
    
    print(f"\n✓ Gap removal complete")
    print(f"  Train: {len(train_df)} records")
    print(f"  Val: {len(val_df)} records")
    print(f"  Test: {len(test_df)} records")


INFO - 
INFO - REMOVING TEMPORAL GAPS (TRAIN ONLY)
INFO - ============================================================
INFO - Analyzing temporal gaps (chunked)...
INFO -   Processed 50000/173448 clients
INFO -   Processed 100000/173448 clients
INFO -   Processed 150000/173448 clients
INFO - Gaps: 1131 clients (0.7%)
INFO - ✓ Removed 1131 clients with gaps
INFO -   Records: 2162862 → 2153407
INFO - 
Removing 1131 clients from val/test as well...
INFO -   Val: 531880 records remaining
INFO -   Test: 542902 records remaining

✓ Gap removal complete
  Train: 2153407 records
  Val: 531880 records
  Test: 542902 records


In [9]:
# Cell 9: Preprocessing Pipeline - Core Methods (NO ENCODING for CatBoost!)
class PreprocessingPipeline:
    """Main preprocessing pipeline - optimized for CatBoost"""
    
    def __init__(self, config):
        self.config = config
        self.logger = logging.getLogger(self.__class__.__name__)
        self.metadata = {
            'config': vars(config),
            'steps': [],
            'timestamp': datetime.now().isoformat()
        }
        self.fitted_columns = None
    
    def fit_transform(self, train_df):
        """Fit preprocessing on training data and transform it"""
        self.logger.info("="*60)
        self.logger.info("FITTING PREPROCESSING PIPELINE")
        self.logger.info("="*60)
        
        df_processed = train_df
        original_shape = df_processed.shape
        self.metadata['original_train_shape'] = original_shape
        
        # Store original column order (before any transformations)
        self.fitted_columns = [col for col in df_processed.columns 
                              if col not in config.ID_COLUMNS + [config.TARGET_COLUMN]]
        
        # Step 1: Remove constant columns
        df_processed, _ = self._remove_constant_columns(df_processed, fit=True)
        
        # Step 2: Handle outliers
        df_processed, _ = self._handle_outliers(df_processed, fit=True)
        
        # Step 3: Handle missing values (NO ENCODING - CatBoost handles categoricals!)
        df_processed, _ = self._handle_missing_values(df_processed, fit=True)
        
        # Step 4: Remove high correlations
        df_processed, _ = self._remove_correlations(df_processed, fit=True)
        
        df_final = df_processed.copy()
        
        # Store final feature list after all transformations
        self.final_features = [col for col in df_final.columns 
                              if col not in config.ID_COLUMNS + [config.TARGET_COLUMN]]
        
        self._log_summary(original_shape, df_final.shape, 'TRAIN')
        
        if df_processed is not train_df and df_processed is not df_final:
            del df_processed
            import gc
            gc.collect()
        
        return df_final
    
    def transform(self, df, dataset_name='test'):
        """Transform new data using fitted preprocessing"""
        self.logger.info(f"="*60)
        self.logger.info(f"TRANSFORMING {dataset_name.upper()} DATA")
        self.logger.info(f"="*60)
        
        df_processed = df
        original_shape = df_processed.shape
        
        df_processed, _ = self._remove_constant_columns(df_processed, fit=False)
        df_processed, _ = self._handle_outliers(df_processed, fit=False)
        df_processed, _ = self._handle_missing_values(df_processed, fit=False)
        df_processed, _ = self._remove_correlations(df_processed, fit=False)
        
        # FINAL: Align columns with training data
        df_processed = self._align_columns_with_training(df_processed, dataset_name)
        
        df_final = df_processed.copy()
        
        self._log_summary(original_shape, df_final.shape, dataset_name)
        
        if df_processed is not df and df_processed is not df_final:
            del df_processed
            import gc
            gc.collect()
        
        return df_final
    
    def _align_columns_with_training(self, df, dataset_name):
        """Ensure DataFrame has same columns as training data"""
        if self.final_features is None:
            self.logger.warning("⚠ No fitted features found, skipping alignment")
            return df
        
        preserve_cols = [col for col in config.ID_COLUMNS if col in df.columns]
        if config.TARGET_COLUMN in df.columns:
            preserve_cols.append(config.TARGET_COLUMN)
        
        current_features = [col for col in df.columns if col not in preserve_cols]
        
        missing_cols = [col for col in self.final_features if col not in current_features]
        extra_cols = [col for col in current_features if col not in self.final_features]
        
        needs_modification = len(missing_cols) > 0 or len(extra_cols) > 0
        
        if needs_modification:
            df = df.copy()
            
            if missing_cols:
                self.logger.warning(f"⚠ Adding {len(missing_cols)} missing columns with zeros")
                for col in missing_cols:
                    df[col] = 0
            
            if extra_cols:
                self.logger.warning(f"⚠ Removing {len(extra_cols)} extra columns")
                df = df.drop(columns=extra_cols)
        
        final_column_order = preserve_cols + self.final_features
        df = df[[col for col in final_column_order if col in df.columns]]
        
        self.logger.info(f"✓ Final columns aligned: {len(df.columns)} total")
        
        return df
    
    def _log_summary(self, original_shape, final_shape, name):
        """Log preprocessing summary"""
        self.logger.info(f"\n{'='*60}")
        self.logger.info(f"SUMMARY - {name}")
        self.logger.info(f"{'='*60}")
        self.logger.info(f"Original: {original_shape}")
        self.logger.info(f"Final: {final_shape}")
        self.logger.info(f"Records: {(1-final_shape[0]/original_shape[0])*100:.1f}% reduction")
        self.logger.info(f"Features: {(1-final_shape[1]/original_shape[1])*100:.1f}% reduction")

print("\n✓ PreprocessingPipeline class defined (optimized for CatBoost)")



✓ PreprocessingPipeline class defined (optimized for CatBoost)


In [10]:
# Cell 10: Preprocessing Pipeline - Helper Methods
def _remove_constant_columns(self, df, fit=True):
    """Remove constant columns"""
    if fit:
        self.logger.info("Step: Removing constant columns...")
        self.constant_cols = []
        
        for col in df.columns:
            if col in config.ID_COLUMNS + [config.TARGET_COLUMN]:
                continue
            n_unique = df[col].nunique(dropna=False)
            if n_unique == 1:
                self.constant_cols.append(col)
        
        if self.constant_cols:
            self.logger.info(f"  Removing {len(self.constant_cols)} constant columns")
            df = df.drop(columns=self.constant_cols)
        else:
            self.logger.info("  No constant columns found")
        
        self.metadata['steps'].append({
            'step': 'remove_constants',
            'n_removed': len(self.constant_cols)
        })
    else:
        self.logger.info(f"  Skipping constant column removal (will align at end)")
    
    return df, {}

def _handle_outliers(self, df, fit=True):
    """Handle outliers in numerical columns"""
    if not self.config.HANDLE_OUTLIERS:
        return df, {}
    
    self.logger.info("Step: Handling outliers...")
    
    if fit:
        target_cols = [col for col in df.columns 
                      if any(kw in col.lower() for kw in 
                            self.config.FINANCIAL_KEYWORDS + self.config.BALANCE_KEYWORDS)
                      and col not in config.ID_COLUMNS + [config.TARGET_COLUMN] + config.CATEGORICAL_FEATURES]
        
        self.outlier_bounds = {}
        for col in target_cols:
            if col in df.columns and df[col].dtype in ['float64', 'float32', 'int64', 'int32', 'int16', 'int8']:
                Q1 = df[col].quantile(0.25)
                Q3 = df[col].quantile(0.75)
                IQR = Q3 - Q1
                self.outlier_bounds[col] = {
                    'lower': Q1 - self.config.OUTLIER_IQR_MULTIPLIER * IQR,
                    'upper': Q3 + self.config.OUTLIER_IQR_MULTIPLIER * IQR
                }
        
        for col, bounds in self.outlier_bounds.items():
            df[col] = df[col].clip(lower=bounds['lower'], upper=bounds['upper'])
        
        self.logger.info(f"  Treated outliers in {len(self.outlier_bounds)} columns")
        
        self.metadata['steps'].append({
            'step': 'handle_outliers',
            'n_columns': len(self.outlier_bounds)
        })
    else:
        n_clipped = 0
        for col, bounds in self.outlier_bounds.items():
            if col in df.columns:
                df[col] = df[col].clip(lower=bounds['lower'], upper=bounds['upper'])
                n_clipped += 1
        
        self.logger.info(f"  Applied outlier clipping to {n_clipped}/{len(self.outlier_bounds)} columns")
    
    return df, {}

def _handle_missing_values(self, df, fit=True):
    """Handle missing values - NO ENCODING for CatBoost!"""
    self.logger.info("Step: Handling missing values...")
    
    if fit:
        # Separate numeric and categorical (but don't encode categorical!)
        self.numeric_cols_for_imputation = [col for col in df.select_dtypes(include=[np.number]).columns
                                           if col not in config.ID_COLUMNS + [config.TARGET_COLUMN]]
        
        # CatBoost handles categorical features natively - just keep them as strings
        self.categorical_cols_for_imputation = [col for col in config.CATEGORICAL_FEATURES 
                                               if col in df.columns]
        
        # Fit imputers
        self.numeric_imputer = SimpleImputer(strategy='median')
        self.categorical_imputer = SimpleImputer(strategy='most_frequent')
        
        if len(self.numeric_cols_for_imputation) > 0:
            df[self.numeric_cols_for_imputation] = self.numeric_imputer.fit_transform(
                df[self.numeric_cols_for_imputation]
            )
        
        if len(self.categorical_cols_for_imputation) > 0:
            df[self.categorical_cols_for_imputation] = self.categorical_imputer.fit_transform(
                df[self.categorical_cols_for_imputation]
            )
        
        self.logger.info(f"  Imputed: {len(self.numeric_cols_for_imputation)} numeric, "
                        f"{len(self.categorical_cols_for_imputation)} categorical")
        self.logger.info(f"  ℹ Categorical features kept as-is for CatBoost")
        
        self.metadata['steps'].append({
            'step': 'handle_missing',
            'n_numeric': len(self.numeric_cols_for_imputation),
            'n_categorical': len(self.categorical_cols_for_imputation),
            'note': 'No encoding - CatBoost handles categoricals natively'
        })
    else:
        numeric_cols_present = [col for col in self.numeric_cols_for_imputation if col in df.columns]
        categorical_cols_present = [col for col in self.categorical_cols_for_imputation if col in df.columns]
        
        missing_numeric = [col for col in self.numeric_cols_for_imputation if col not in df.columns]
        if missing_numeric:
            self.logger.warning(f"⚠ Adding {len(missing_numeric)} missing numeric columns with fill values")
            for i, col in enumerate(missing_numeric):
                fill_value = self.numeric_imputer.statistics_[
                    self.numeric_cols_for_imputation.index(col)
                ]
                df[col] = fill_value
        
        missing_categorical = [col for col in self.categorical_cols_for_imputation if col not in df.columns]
        if missing_categorical:
            self.logger.warning(f"⚠ Adding {len(missing_categorical)} missing categorical columns with fill values")
            for i, col in enumerate(missing_categorical):
                fill_value = self.categorical_imputer.statistics_[
                    self.categorical_cols_for_imputation.index(col)
                ]
                df[col] = fill_value
        
        if len(self.numeric_cols_for_imputation) > 0 and hasattr(self, 'numeric_imputer'):
            df[self.numeric_cols_for_imputation] = self.numeric_imputer.transform(
                df[self.numeric_cols_for_imputation]
            )
        
        if len(self.categorical_cols_for_imputation) > 0 and hasattr(self, 'categorical_imputer'):
            df[self.categorical_cols_for_imputation] = self.categorical_imputer.transform(
                df[self.categorical_cols_for_imputation]
            )
    
    return df, {}

def _remove_correlations(self, df, fit=True):
    """Remove highly correlated features"""
    if not self.config.REMOVE_HIGH_CORRELATIONS:
        return df, {}
    
    self.logger.info("Step: Removing high correlations...")
    
    if fit:
        # Only numeric, non-categorical columns
        numeric_cols = [col for col in df.select_dtypes(include=[np.number]).columns
                       if col not in config.ID_COLUMNS + [config.TARGET_COLUMN] + config.CATEGORICAL_FEATURES]
        
        if len(numeric_cols) > 1:
            corr_matrix = df[numeric_cols].corr().abs()
            upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
            
            self.features_to_drop_corr = [col for col in upper.columns 
                                         if any(upper[col] > self.config.CORRELATION_THRESHOLD)]
            
            if self.features_to_drop_corr:
                df = df.drop(columns=self.features_to_drop_corr)
                self.logger.info(f"  Removed {len(self.features_to_drop_corr)} correlated features")
            else:
                self.logger.info("  No highly correlated features found")
        else:
            self.features_to_drop_corr = []
        
        self.metadata['steps'].append({
            'step': 'remove_correlations',
            'n_removed': len(self.features_to_drop_corr)
        })
    else:
        self.logger.info(f"  Skipping correlation removal (will align at end)")
    
    return df, {}

# Attach helper methods to PreprocessingPipeline class
PreprocessingPipeline._remove_constant_columns = _remove_constant_columns
PreprocessingPipeline._handle_outliers = _handle_outliers
PreprocessingPipeline._handle_missing_values = _handle_missing_values
PreprocessingPipeline._remove_correlations = _remove_correlations

print("\n✓ PreprocessingPipeline helper methods attached")



✓ PreprocessingPipeline helper methods attached


In [11]:
# Cell 11: Fit Preprocessing Pipeline on Training Data
logger.info("\n" + "="*60)
logger.info("FITTING PREPROCESSING ON TRAINING DATA")
logger.info("="*60)

# Initialize pipeline
pipeline = PreprocessingPipeline(config)

# Fit and transform training data
train_processed = pipeline.fit_transform(train_df)

print(f"\n✓ Preprocessing fitted on training data")
print(f"  Original shape: {train_df.shape}")
print(f"  Processed shape: {train_processed.shape}")
print(f"  Features stored: {len(pipeline.final_features)}")

# Clean up
del train_df
import gc
gc.collect()

print(f"  Memory cleaned")

INFO - 
INFO - FITTING PREPROCESSING ON TRAINING DATA
INFO - ============================================================
INFO - ============================================================
INFO - FITTING PREPROCESSING PIPELINE
INFO - ============================================================
INFO - Step: Removing constant columns...
INFO -   Removing 9 constant columns
INFO - Step: Handling outliers...
INFO -   Treated outliers in 91 columns
INFO - Step: Handling missing values...
INFO -   Imputed: 181 numeric, 3 categorical
INFO -   ℹ Categorical features kept as-is for CatBoost
INFO - Step: Removing high correlations...
INFO -   Removed 61 correlated features
INFO - 
INFO - SUMMARY - TRAIN
INFO - ============================================================
INFO - Original: (2153407, 195)
INFO - Final: (2153407, 125)
INFO - Records: 0.0% reduction
INFO - Features: 35.9% reduction

✓ Preprocessing fitted on training data
  Original shape: (2153407, 195)
  Processed shape: (2153407, 

In [12]:
# Cell 12: Transform Validation and Test Data
logger.info("\n" + "="*60)
logger.info("TRANSFORMING VALIDATION AND TEST DATA")
logger.info("="*60)

# Transform validation data
val_processed = pipeline.transform(val_df, dataset_name='validation')

# Transform test data
test_processed = pipeline.transform(test_df, dataset_name='test')

print(f"\n✓ Validation and test data transformed")
print(f"  Validation: {val_df.shape} → {val_processed.shape}")
print(f"  Test: {test_df.shape} → {test_processed.shape}")

# Clean up
del val_df, test_df
import gc
gc.collect()

print(f"  Memory cleaned")

INFO - 
INFO - TRANSFORMING VALIDATION AND TEST DATA
INFO - ============================================================
INFO - ============================================================
INFO - TRANSFORMING VALIDATION DATA
INFO - ============================================================
INFO -   Skipping constant column removal (will align at end)
INFO - Step: Handling outliers...
INFO -   Applied outlier clipping to 91/91 columns
INFO - Step: Handling missing values...
INFO - Step: Removing high correlations...
INFO -   Skipping correlation removal (will align at end)
WARNING - ⚠ Removing 70 extra columns
INFO - ✓ Final columns aligned: 125 total
INFO - 
INFO - SUMMARY - validation
INFO - ============================================================
INFO - Original: (531880, 195)
INFO - Final: (531880, 125)
INFO - Records: 0.0% reduction
INFO - Features: 35.9% reduction
INFO - ============================================================
INFO - TRANSFORMING TEST DATA
INFO - =======

In [13]:
# Cell 13: Load and Transform Production Data
logger.info("\n" + "="*60)
logger.info("LOADING PRODUCTION DATA")
logger.info("="*60)

prod_df = None
prod_path = config.get_prod_input_path()

if prod_path.exists():
    prod_loader = DataLoader(prod_path, delimiter=config.DELIMITER)
    prod_df = prod_loader.load()
    
    logger.info("✓ Production data loaded")
    logger.info("ℹ Production data is for scoring (no target)")
    
    print(f"\n✓ Production data loaded: {prod_df.shape}")
    
    # Transform production data
    logger.info("\n" + "="*60)
    logger.info("TRANSFORMING PRODUCTION DATA")
    logger.info("="*60)
    
    try:
        prod_processed = pipeline.transform(prod_df, dataset_name='production')
        
        print(f"✓ Production data transformed")
        print(f"  Original: {prod_df.shape}")
        print(f"  Processed: {prod_processed.shape}")
        
    except Exception as e:
        logger.error(f"✗ Error transforming production data: {e}")
        print(f"\n✗ Error transforming production data: {e}")
        print("\nℹ Check that production data has compatible features")
        prod_processed = None

INFO - 
INFO - LOADING PRODUCTION DATA
INFO - ============================================================
INFO - Loading data from: data\churn_prod_ul.csv
INFO -   Memory before optimization: 329.52 MB
INFO -   Memory after optimization: 144.16 MB (56.3% saved)
INFO - ✓ Data loaded: (206770, 194)
INFO - ✓ Data validation passed (no target - production data)
INFO - ✓ Production data loaded
INFO - ℹ Production data is for scoring (no target)

✓ Production data loaded: (206770, 194)
INFO - 
INFO - TRANSFORMING PRODUCTION DATA
INFO - ============================================================
INFO - ============================================================
INFO - TRANSFORMING PRODUCTION DATA
INFO - ============================================================
INFO -   Skipping constant column removal (will align at end)
INFO - Step: Handling outliers...
INFO -   Applied outlier clipping to 91/91 columns
INFO - Step: Handling missing values...
INFO - Step: Removing high correlations...


In [14]:
# Cell 14: Save All Processed Data
logger.info("\n" + "="*60)
logger.info("SAVING PROCESSED DATA")
logger.info("="*60)

# Save datasets
train_path = config.OUTPUT_DIR / "train_processed.csv"
val_path = config.OUTPUT_DIR / "val_processed.csv"
test_path = config.OUTPUT_DIR / "test_processed.csv"

train_processed.to_csv(train_path, index=False, sep='|')
val_processed.to_csv(val_path, index=False, sep='|')
test_processed.to_csv(test_path, index=False, sep='|')

logger.info(f"✓ Saved training data: {train_path}")
logger.info(f"✓ Saved validation data: {val_path}")
logger.info(f"✓ Saved test data: {test_path}")

print(f"\n✓ Processed data saved:")
print(f"  Train: {train_path}")
print(f"  Val: {val_path}")
print(f"  Test: {test_path}")

if prod_processed is not None:
    prod_path = config.OUTPUT_DIR / "prod_processed.csv"
    prod_processed.to_csv(prod_path, index=False, sep='|')
    logger.info(f"✓ Saved production data: {prod_path}")
    print(f"  Prod: {prod_path}")

# Save metadata
metadata_path = config.OUTPUT_DIR / config.METADATA_FILE
pipeline.metadata['split_info'] = split_info
pipeline.metadata['categorical_features'] = config.CATEGORICAL_FEATURES
with open(metadata_path, 'w') as f:
    json.dump(pipeline.metadata, f, indent=2, default=str)

logger.info(f"✓ Saved metadata: {metadata_path}")
print(f"\n✓ Metadata saved: {metadata_path}")

# Save categorical features list for model training
cat_features_path = config.OUTPUT_DIR / "categorical_features.json"
with open(cat_features_path, 'w') as f:
    json.dump(config.CATEGORICAL_FEATURES, f, indent=2)
logger.info(f"✓ Saved categorical features list: {cat_features_path}")
print(f"✓ Categorical features list saved: {cat_features_path}")

INFO - 
INFO - SAVING PROCESSED DATA
INFO - ============================================================
INFO - ✓ Saved training data: output\train_processed.csv
INFO - ✓ Saved validation data: output\val_processed.csv
INFO - ✓ Saved test data: output\test_processed.csv

✓ Processed data saved:
  Train: output\train_processed.csv
  Val: output\val_processed.csv
  Test: output\test_processed.csv
INFO - ✓ Saved production data: output\prod_processed.csv
  Prod: output\prod_processed.csv
INFO - ✓ Saved metadata: output\preprocessing_metadata.json

✓ Metadata saved: output\preprocessing_metadata.json
INFO - ✓ Saved categorical features list: output\categorical_features.json
✓ Categorical features list saved: output\categorical_features.json


In [15]:
# Cell 17: Final Summary and Verification
logger.info("\n" + "="*60)
logger.info("PREPROCESSING PIPELINE COMPLETED")
logger.info("="*60)

print("\n" + "="*60)
print("✓ PREPROCESSING COMPLETE - SUMMARY")
print("="*60)

print(f"\nFinal Dataset Shapes:")
print(f"  Training:   {train_processed.shape}")
print(f"  Validation: {val_processed.shape}")
print(f"  Test:       {test_processed.shape}")
if prod_processed is not None:
    print(f"  Production: {prod_processed.shape}")

if config.TARGET_COLUMN in train_processed.columns:
    print(f"\nChurn Rates:")
    print(f"  Training:   {train_processed[config.TARGET_COLUMN].mean():.4f}")
    print(f"  Validation: {val_processed[config.TARGET_COLUMN].mean():.4f}")
    print(f"  Test:       {test_processed[config.TARGET_COLUMN].mean():.4f}")

print(f"\nOutput Files:")
print(f"  Data: {config.OUTPUT_DIR}")
print(f"  Logs: {config.LOG_DIR}")

print("\n" + "="*60)
print("✓ READY FOR MODEL TRAINING")
print("="*60)

print("\nNext Steps:")
print("1. Review logs in the 'logs' directory")
print("2. Check processed data in 'output' directory")
print("3. Train model on train_processed")
print("4. Tune on val_processed")
print("5. Evaluate on test_processed")
print("6. Score on prod_processed (if available)")

INFO - 
INFO - PREPROCESSING PIPELINE COMPLETED
INFO - ============================================================

✓ PREPROCESSING COMPLETE - SUMMARY

Final Dataset Shapes:
  Training:   (2153407, 125)
  Validation: (531880, 125)
  Test:       (542902, 125)
  Production: (206770, 124)

Churn Rates:
  Training:   0.0144
  Validation: 0.0177
  Test:       0.0145

Output Files:
  Data: output
  Logs: logs

✓ READY FOR MODEL TRAINING

Next Steps:
1. Review logs in the 'logs' directory
2. Check processed data in 'output' directory
3. Train model on train_processed
4. Tune on val_processed
5. Evaluate on test_processed
6. Score on prod_processed (if available)


In [16]:
# ============================================================
# CELL 18: Multi-Model & Multi-Segment Configuration
# ============================================================

import pickle
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

class MultiModelConfig:
    """Configuration for multi-model, multi-segment training"""
    
    # Directories
    SEGMENT_DIR = config.OUTPUT_DIR / "segments"
    MODEL_SPECIFIC_DIR = config.OUTPUT_DIR / "model_specific"
    ENCODERS_DIR = config.OUTPUT_DIR / "encoders"
    
    # Segments
    SEGMENTS = ['SMALL_BUSINESS', 'MIDDLE_BUSINESS', 'LARGE_BUSINESS']
    SEGMENT_COLUMN = 'segment_group'
    
    # Models
    MODELS = ['catboost', 'xgboost', 'lightgbm']
    
    # Encoding strategies
    ENCODING_STRATEGY = {
        'catboost': 'none',      # CatBoost handles categoricals natively
        'xgboost': 'onehot',     # XGBoost needs one-hot encoding
        'lightgbm': 'label'      # LightGBM works best with label encoding
    }
    
    @classmethod
    def create_directories(cls):
        """Create all necessary directories"""
        for dir_path in [cls.SEGMENT_DIR, cls.MODEL_SPECIFIC_DIR, cls.ENCODERS_DIR]:
            dir_path.mkdir(parents=True, exist_ok=True)
        
        for model in cls.MODELS:
            (cls.MODEL_SPECIFIC_DIR / model).mkdir(parents=True, exist_ok=True)

# Initialize
multi_config = MultiModelConfig()
multi_config.create_directories()

logger.info("\n" + "="*60)
logger.info("MULTI-MODEL MULTI-SEGMENT CONFIGURATION")
logger.info("="*60)
logger.info(f"Segments: {multi_config.SEGMENTS}")
logger.info(f"Models: {multi_config.MODELS}")

print("\n✓ Multi-model configuration initialized")
print(f"  Segments: {multi_config.SEGMENTS}")
print(f"  Models: {multi_config.MODELS}")

INFO - 
INFO - MULTI-MODEL MULTI-SEGMENT CONFIGURATION
INFO - ============================================================
INFO - Segments: ['SMALL_BUSINESS', 'MIDDLE_BUSINESS', 'LARGE_BUSINESS']
INFO - Models: ['catboost', 'xgboost', 'lightgbm']

✓ Multi-model configuration initialized
  Segments: ['SMALL_BUSINESS', 'MIDDLE_BUSINESS', 'LARGE_BUSINESS']
  Models: ['catboost', 'xgboost', 'lightgbm']


In [17]:
# ============================================================
# CELL 19: Multi-Model Encoder Class
# ============================================================

class MultiModelEncoder:
    """Handle encoding for different ML frameworks"""
    
    def __init__(self, categorical_features):
        self.categorical_features = categorical_features
        self.label_encoders = {}
        self.onehot_encoder = None
        self.onehot_feature_names = []
        self.logger = logging.getLogger(self.__class__.__name__)
    
    def fit_label_encoding(self, df):
        """Fit label encoders for LightGBM"""
        self.logger.info("  Fitting Label Encoders for LightGBM...")
        
        for col in self.categorical_features:
            if col in df.columns:
                le = LabelEncoder()
                le.fit(df[col].astype(str))
                self.label_encoders[col] = le
        
        self.logger.info(f"    ✓ Fitted {len(self.label_encoders)} label encoders")
        return self
    
    def transform_label_encoding(self, df):
        """Apply label encoding for LightGBM"""
        df_encoded = df.copy()
        
        for col, encoder in self.label_encoders.items():
            if col in df_encoded.columns:
                df_encoded[col] = df_encoded[col].astype(str).apply(
                    lambda x: encoder.transform([x])[0] if x in encoder.classes_ else -1
                )
        
        return df_encoded
    
    def fit_onehot_encoding(self, df):
        """Fit one-hot encoder for XGBoost"""
        self.logger.info("  Fitting One-Hot Encoder for XGBoost...")
        
        categorical_data = df[self.categorical_features].astype(str)
        
        self.onehot_encoder = OneHotEncoder(
            drop='first',
            sparse_output=False,
            handle_unknown='ignore'
        )
        
        self.onehot_encoder.fit(categorical_data)
        self.onehot_feature_names = self.onehot_encoder.get_feature_names_out(
            self.categorical_features
        ).tolist()
        
        self.logger.info(f"    ✓ One-hot encoding creates {len(self.onehot_feature_names)} features")
        return self
    
    def transform_onehot_encoding(self, df):
        """Apply one-hot encoding for XGBoost"""
        df_encoded = df.copy()
        categorical_data = df_encoded[self.categorical_features].astype(str)
        encoded_array = self.onehot_encoder.transform(categorical_data)
        
        encoded_df = pd.DataFrame(
            encoded_array,
            columns=self.onehot_feature_names,
            index=df_encoded.index
        )
        
        df_encoded = df_encoded.drop(columns=self.categorical_features)
        df_encoded = pd.concat([df_encoded, encoded_df], axis=1)
        
        return df_encoded
    
    def prepare_for_model(self, df, model_type, fit=False):
        """Prepare data for specific model type"""
        
        if model_type == 'catboost':
            return df.copy()
        
        elif model_type == 'lightgbm':
            if fit:
                self.fit_label_encoding(df)
            return self.transform_label_encoding(df)
        
        elif model_type == 'xgboost':
            if fit:
                self.fit_onehot_encoding(df)
            return self.transform_onehot_encoding(df)
        
        else:
            raise ValueError(f"Unknown model type: {model_type}")
    
    def save(self, filepath):
        """Save encoders to file"""
        with open(filepath, 'wb') as f:
            pickle.dump({
                'label_encoders': self.label_encoders,
                'onehot_encoder': self.onehot_encoder,
                'onehot_feature_names': self.onehot_feature_names,
                'categorical_features': self.categorical_features
            }, f)

print("✓ MultiModelEncoder class defined")

✓ MultiModelEncoder class defined


In [18]:
# ============================================================
# CELL 20: Segment Distribution Analysis
# ============================================================

logger.info("\n" + "="*60)
logger.info("SEGMENT DISTRIBUTION ANALYSIS")
logger.info("="*60)

print("\n" + "="*60)
print("SEGMENT DISTRIBUTION ANALYSIS")
print("="*60)

# Analyze each dataset
for dataset_name, df in [('Train', train_processed), ('Val', val_processed), ('Test', test_processed)]:
    print(f"\n{dataset_name}:")
    
    for segment in multi_config.SEGMENTS:
        segment_df = df[df[multi_config.SEGMENT_COLUMN] == segment]
        churn_count = segment_df[config.TARGET_COLUMN].sum()
        churn_rate = segment_df[config.TARGET_COLUMN].mean()
        imbalance = (1 - churn_rate) / churn_rate if churn_rate > 0 else 0
        
        print(f"  {segment:20s}: {len(segment_df):7,} records | "
              f"Churn: {churn_count:5,} ({churn_rate*100:5.2f}%) | "
              f"Imbalance: 1:{imbalance:.1f}")
        
        logger.info(f"{dataset_name} - {segment}: {len(segment_df):,} records, "
                   f"churn rate: {churn_rate:.4f}")

print("\n⚠ Note: Severe class imbalance - will handle in training notebook")

INFO - 
INFO - SEGMENT DISTRIBUTION ANALYSIS
INFO - ============================================================

SEGMENT DISTRIBUTION ANALYSIS

Train:
  SMALL_BUSINESS      : 2,024,010 records | Churn: 30,472 ( 1.51%) | Imbalance: 1:65.4
INFO - Train - SMALL_BUSINESS: 2,024,010 records, churn rate: 0.0151
  MIDDLE_BUSINESS     : 115,095 records | Churn:   550 ( 0.48%) | Imbalance: 1:208.3
INFO - Train - MIDDLE_BUSINESS: 115,095 records, churn rate: 0.0048
  LARGE_BUSINESS      :  14,302 records | Churn:    92 ( 0.64%) | Imbalance: 1:154.5
INFO - Train - LARGE_BUSINESS: 14,302 records, churn rate: 0.0064

Val:
  SMALL_BUSINESS      : 493,087 records | Churn: 9,157 ( 1.86%) | Imbalance: 1:52.8
INFO - Val - SMALL_BUSINESS: 493,087 records, churn rate: 0.0186
  MIDDLE_BUSINESS     :  35,200 records | Churn:   255 ( 0.72%) | Imbalance: 1:137.0
INFO - Val - MIDDLE_BUSINESS: 35,200 records, churn rate: 0.0072
  LARGE_BUSINESS      :   3,593 records | Churn:    21 ( 0.58%) | Imbalance: 1:170.

In [19]:
# ============================================================
# CELL 21: Split Data by Segments
# ============================================================

logger.info("\n" + "="*60)
logger.info("SPLITTING DATA BY SEGMENTS")
logger.info("="*60)

segment_datasets = {}

for segment in multi_config.SEGMENTS:
    logger.info(f"\nProcessing segment: {segment}")
    
    # Filter each dataset by segment
    segment_train = train_processed[train_processed[multi_config.SEGMENT_COLUMN] == segment].copy()
    segment_val = val_processed[val_processed[multi_config.SEGMENT_COLUMN] == segment].copy()
    segment_test = test_processed[test_processed[multi_config.SEGMENT_COLUMN] == segment].copy()
    
    segment_datasets[segment] = {
        'train': segment_train,
        'val': segment_val,
        'test': segment_test
    }
    
    logger.info(f"  Train: {len(segment_train):,} records")
    logger.info(f"  Val: {len(segment_val):,} records")
    logger.info(f"  Test: {len(segment_test):,} records")
    
    # Save raw segment splits
    segment_dir = multi_config.SEGMENT_DIR / segment
    segment_dir.mkdir(parents=True, exist_ok=True)
    
    segment_train.to_csv(segment_dir / "train_raw.csv", index=False, sep='|')
    segment_val.to_csv(segment_dir / "val_raw.csv", index=False, sep='|')
    segment_test.to_csv(segment_dir / "test_raw.csv", index=False, sep='|')

# Production data by segment
if prod_processed is not None:
    logger.info("\nSplitting production data...")
    for segment in multi_config.SEGMENTS:
        segment_prod = prod_processed[prod_processed[multi_config.SEGMENT_COLUMN] == segment].copy()
        segment_datasets[segment]['prod'] = segment_prod
        
        segment_dir = multi_config.SEGMENT_DIR / segment
        segment_prod.to_csv(segment_dir / "prod_raw.csv", index=False, sep='|')
        logger.info(f"  {segment}: {len(segment_prod):,} prod records")

print(f"\n✓ Data split by segments: {multi_config.SEGMENT_DIR}")

INFO - 
INFO - SPLITTING DATA BY SEGMENTS
INFO - ============================================================
INFO - 
Processing segment: SMALL_BUSINESS
INFO -   Train: 2,024,010 records
INFO -   Val: 493,087 records
INFO -   Test: 500,370 records
INFO - 
Processing segment: MIDDLE_BUSINESS
INFO -   Train: 115,095 records
INFO -   Val: 35,200 records
INFO -   Test: 39,032 records
INFO - 
Processing segment: LARGE_BUSINESS
INFO -   Train: 14,302 records
INFO -   Val: 3,593 records
INFO -   Test: 3,500 records
INFO - 
Splitting production data...
INFO -   SMALL_BUSINESS: 191,107 prod records
INFO -   MIDDLE_BUSINESS: 14,508 prod records
INFO -   LARGE_BUSINESS: 1,155 prod records

✓ Data split by segments: output\segments
